In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets

from ipywidgets import interact, interactive
from datetime import timedelta, datetime
import folium,json

Populating the interactive namespace from numpy and matplotlib


C:\Users\Alexey\Anaconda2\lib\site-packages\folium\__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [2]:
df0=pd.read_csv('final_data.csv',sep=',',parse_dates=['t'],index_col=['t'])
cols=df0.columns
r_cols=list(set([x[:4]for x in df0.columns]))

In [3]:
def get_dfs():
    dfs_dic={}
    for step in [1,3,6]:
        yr_name=[x+'y_'+str(step) for x in r_cols]
        ypr_abs_name=[x+'ypr_abs_'+str(step) for x in r_cols]
        df_real=df0[yr_name]
        df_predicted=df0[ypr_abs_name]
        df_real.columns=r_cols
        df_predicted.columns=r_cols
        
        dfs_dic[step]=[df_real,df_predicted]
    
    return  dfs_dic

In [4]:
#Блок подготовки карт
NY_center_location=[(40.49612+40.91553)/2+0.04,(-74.25559+-73.70001)/2+0.2] #Координаты центра NY
south,north,west,east=40.49612, 40.91553,-74.25559, -73.70001
coord_NY=[[south,west],[south,east],[north,east],[north,west],[south,west]] 
NY_border=folium.Polygon(coord_NY,color='blue',weight=0.8)

#Создаем df с координатами 102 регионов
initial_regs=pd.read_csv('regions.csv',sep=';')
initial_regs.index=initial_regs.region #индексы делаем равными идентификаторам регионов
indexes_r=[int(x) for x in r_cols] #выделяем индексы, соответствующие 102 регионам
regs=initial_regs.loc[indexes_r] # датафрейм с координатами 102 регионов
indexes_r_sorted=np.sort(indexes_r)
indexes_r_sorted_str=[str(x) for x in indexes_r_sorted]
map_reg_state=pd.DataFrame(index=range(1,103))
map_reg_state['id']=indexes_r_sorted_str
map_reg_state[0]=0

#Создаем полигоны 102 ячеек для последующего перевода в json формат
Polygons=[]

#map_bins=[0,50,100,500,1500,2500]
map_bins=[0,100,500,1000,2500]

for i in regs.index[:]:
    w,e,s,n=regs.west[i],regs.east[i],regs.south[i],regs.north[i]
    coord=[[[w,s],[e,s],[e,n],[w,n],[w,s]]] #координаты полигона ячейки
    D={"type":"Feature",0:"AL","geometry":{"type":"Polygon","coordinates":0}}
    D['id']=str(i)
    D["geometry"]["coordinates"]=coord
    Polygons.append(D)
    
form={"type":"FeatureCollection","features":Polygons} #Формирование геоданных
NYgeo = json.dumps(form) #Перевод в json формат

dfs_dic=get_dfs()

In [5]:
def get_map(step,t1):#Рисует карту с цветовым отображением числа поездок в час
    df_pr=dfs_dic[step][1][t1:t1].T
    df_pr.columns=[0]
    df_pr['id']=df_pr.index

    m1 = folium.Map(location=NY_center_location, zoom_start=10.5,height='80%', width='90%')

    folium.Choropleth(
        geo_data=NYgeo,
        name='choropleth',
        data=df_pr,
        columns=['id', 0],
        key_on='feature.id', #соединяем геоданные с данными числа поездок по полю 'id'
        bins=map_bins,
        fill_color='Oranges',
        fill_opacity=0.8,
        line_opacity=0.3,
        legend_name=u'Среднее количество поездок в час'
    ).add_to(m1)
    
    NY_border.add_to(m1)

    return m1

In [6]:
def get_map00(map_reg_state,opacity):#Рисует карту с цветовым отображением выбора ячейки
    m1 = folium.Map(location=NY_center_location, zoom_start=11,height='90%', width='90%')

    folium.Choropleth(
        geo_data=NYgeo,
        name='choropleth',
        data=map_reg_state,
        columns=['id', 0],
        key_on='feature.id', #соединяем геоданные с данными числа поездок по полю 'id'
        bins=[0,2,4,6],
        fill_color='YlGn',#'Reds',#'Greys',#'Greens',#'RdPu',#'PRGn',
       # fill_color=  {229,245,249},
       # fill_color=colorscale ,
        fill_opacity=opacity,
        line_opacity=0.1,
        #legend_name=u'Среднее количество поездок в час'
    ).add_to(m1)
    
    NY_border.add_to(m1)

    return m1

In [7]:
def get_graph (r,step,t1,t2): # Построение графика числа поездок в час
    df_r=dfs_dic[step][0][r][t1:t2]
    df_p=dfs_dic[step][1][r][t1:t2]
    plt.figure(figsize=(18,5))
    plt.plot(df_r)
    plt.plot(df_p)
    plt.legend(('Real', 'Predicted'),loc='upper right')

In [8]:
#Блок описания и компоновки виджетов

#Месяц начала периода
w_month_start= widgets.RadioButtons(
    description='Month start:',
    options=[ 'May', 'Jun'],
    value='May'
)

day_max_dic={'May':31, 'Jun':30}

#День начала периода
w_day_start=widgets.IntSlider(
    value=1,
    min=1,
    max=day_max_dic[w_month_start.value],
    step=1,
    description='Day start:',)

#Час начала периода
w_hour_start=widgets.IntSlider(
    value=0,
    min=0,
    max=23,
    step=1,
    description='hour start',)
    #orientation='vertical', )

#Блок шага предсказания
w_prediction_step= widgets.RadioButtons(
    description='Prediction on:',
    options=['step=1','step=3', 'step=6', ],
    value='step=3')

step_dic={'step=1':1,'step=2':2,'step=3':3, 'step=4':4,'step=5':5, 'step=6':6}
    
    #Длительность периода в днях
w_day_length=widgets.IntSlider(
    value=0,
    min=0,
    max=45,
    step=1,
    description='in days:',)

#Длительность периода в часах
w_hour_length=widgets.IntSlider(
    value=23,
    min=0,
    max=23,
    step=1,
    description='in hours:',)

#День недели начала и конца
dw1,dw2='',''
w_dw1=widgets.Label('')
w_dw2=widgets.Label('')

# Объединение виджетов дня и часа сначала периода
box_d_h = widgets.VBox(children=[w_day_start, w_hour_start,w_dw1])
box_d = widgets.VBox(children=[w_day_start, w_dw1])

# Объединение виджетов старта
box_start = widgets.HBox(children=[w_month_start, box_d_h, w_prediction_step])
w_4_map = widgets.HBox(children=[w_month_start, box_d, w_prediction_step])

# Объединение виджетов длины периода
l_end=widgets.Label('Length of period:')
box_end1 = widgets.VBox(children=[l_end,w_day_length, w_hour_length])
box_end2 = widgets.HBox(children=[box_end1,w_dw2])

#Полный виджет
w_main= widgets.VBox(children=[box_start,box_end2])

#Объединенный виджет для раскраски карты
box_m_d_step = widgets.HBox(children=[w_month_start, w_day_start, w_prediction_step])
box_h_date = widgets.HBox(children=[w_hour_start, w_dw1])
w_map_color=widgets.VBox(children=[box_m_d_step,box_h_date])

#Блок выбора района
w_reg=widgets.IntSlider(
    value=30,
    min=1,
    max=102,
    step=1,
    description='chose reg:',)

w_opacity=widgets.FloatSlider(
    value=2.,
    min=0.,
    max=2.,
    step=0.1,
    description='opacity:',)



In [9]:
m_dic={'Jan': 1, 'Feb': 2,'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,'Dec 2015': 12, }
y_dic={'Jan': 2016, 'Feb': 2016,'Mar': 2016, 'Apr': 2016, 'May': 2016, 'Jun': 2016,'Dec 2015': 2015, }

def get_time_period(): #вычисляет начальную и конечную даты просматриваемого периода
    year_st=y_dic[w_month_start.value]
    month_st=m_dic[w_month_start.value]
    day_st=w_day_start.value
    hour_st=w_hour_start.value
    day_l=w_day_length.value
    hour_l=w_hour_length.value

    date1=datetime(year_st,month_st,day_st,hour_st)
    delta_date=timedelta(days=day_l,hours=hour_l)
    date2=date1+delta_date
    return str(date1),str(date2)

weekday_dic={0:'MON',1:'TUE',2:'WED',3:'THU',4:'FRI',5:'SAT',6:'SUN'}
def get_days_of_week(t1,t2): # вычисляет дни недели начальной и конечной дат просматриваемого периода
    

    wd1 = weekday_dic[datetime.strptime(t1[:10], "%Y-%m-%d").weekday()]
    wd2 = weekday_dic[datetime.strptime(t2[:10], "%Y-%m-%d").weekday()]
    return wd1,wd2

Виджет отображения цвета ячеек карты в зависимости от числа поездок

In [10]:
def dinamic_map(m1,d1,h1,st,):
   # w_day_start
    t1=get_time_period()[0]
    step=step_dic[st]
    wd1,wd2=get_days_of_week(t1,t1)
    w_dw1.value='date:   '+t1[:13]+', '+wd1
    
    print step
    return display(get_map(step,t1))

   
    
out_m = widgets.interactive_output(dinamic_map, 
    {'m1':w_month_start,
     'd1':w_day_start,
     'h1':w_hour_start,
     'st':w_prediction_step,
    })

Виджет отображения выбора ячейки на карте

In [11]:
def cell_state(opacity,h):
    cell_id=map_reg_state.id[h]
    n=np.zeros(102)
    n[h-1]=5
    map_reg_state[0]=n
        
    print 'Region id :', cell_id
    return display(get_map00(map_reg_state,opacity))

In [12]:
map_cell_state_widg=interactive(cell_state,opacity=w_opacity,h=w_reg,)

Виджет рисования графика числа поездок

In [13]:
def get_graph_widget(m1,d1,h1,d2,h2,l_d, l_h, step, reg): #Обработка виджетов для рисования uhfabrjd
    
 #   w_day_start.max=day_max_dic[w_month_start.value]
    t1,t2= get_time_period()
    wd1,wd2=get_days_of_week(t1,t2)
    w_dw1.value='date:   '+t1[:13]+', '+wd1
    w_dw2.value='date:   '+t2[:13]+', '+wd2
    st=step_dic[step]
    cell_id=map_reg_state.id[reg]
            
    #print t1,step_dic[step]
   
    print 'Region id : ', cell_id
    return get_graph (cell_id,st,t1,t2)
    
    
out_graph = widgets.interactive_output(get_graph_widget, 
    {'m1':w_month_start,'d1':w_day_start,'h1':w_hour_start,
     'd2':w_day_length,'h2':w_hour_length,
    'l_d':w_day_length,'l_h':w_hour_length,
     'step':w_prediction_step,
    'reg':w_reg})

Ваш проект почти закончен!
На этой неделе вам предстоит оформить результат, используя интерактивные графики.

﻿Определитесь, в каком виде вы хотите представить ваш проект. Мы предлагаем вам сделать интерактивное демо, содержащее:

карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени
временной ряд фактического и прогнозируемого спроса на такси в выбираемой области.

## Карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени

### Цветовая динамика числа поездок по районам
#### в зависимости от : месяца, дня, часа и шага предсказания

##### ! Из-за ограничений по объему прилагаемых файлов (<5Мб) для сдачи проекта пришлось урезать исходные данные по максимому: оставлен май, июнь и данные с шагами предсказания 1,3 и 6). В исходной версии все шаги (1-6) 

In [14]:
display(w_map_color,out_m)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oUmFkaW9CdXR0b25zKGRlc2NyaXB0aW9uPXUnTW9udGggc3RhcnQ6Jywgb3B0aW9ucz0oJ01heScsICdKdW4nKSwgdmFsdWU9J01heScpLCDigKY=


T3V0cHV0KG91dHB1dHM9KHt1J291dHB1dF90eXBlJzogdSdzdHJlYW0nLCB1J25hbWUnOiB1J3N0ZG91dCcsIHUndGV4dCc6IHUnM1xuJ30sIHt1J291dHB1dF90eXBlJzogdSdkaXNwbGF5X2TigKY=


## Временной ряд фактического и прогнозируемого спроса на такси в выбираемой области.

### Выберите интересуемую ячейку на карте

In [15]:
display(map_cell_state_widg)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTIuMCwgZGVzY3JpcHRpb249dSdvcGFjaXR5OicsIG1heD0yLjApLCBJbnRTbGlkZXIodmFsdWU9MzAsIGRlc2NyaXB0aW/igKY=


### Для выбранной ячейки представлен график числа поездок в час: предсказанный и фактический в зависимости от шага предсказания

#### Окно периода задается точкой начала (месяц, день, час) и длительностью периода (дни+часы)

In [16]:
display(w_main,out_graph,)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oUmFkaW9CdXR0b25zKGRlc2NyaXB0aW9uPXUnTW9udGggc3RhcnQ6Jywgb3B0aW9ucz0oJ01heScsICdKdW4nKSwgdmFsdWU9J01heScpLCDigKY=


T3V0cHV0KG91dHB1dHM9KHt1J291dHB1dF90eXBlJzogdSdzdHJlYW0nLCB1J25hbWUnOiB1J3N0ZG91dCcsIHUndGV4dCc6IHUnUmVnaW9uIGlkIDogIDEyMjdcbid9LCB7dSdvdXRwdXRfdHnigKY=
